In [16]:
# 📊 Data Fetch + Feature Engineering Cell
%pip install yfinance

import pandas as pd
import numpy as np
from ta import add_all_ta_features
import yfinance as yf
import os

# Step 1: Create data directory
os.makedirs('data', exist_ok=True)

# Step 2: Download SPY data (proxy for /ES futures)
ticker = 'SPY'
df = yf.download(ticker, start='2020-01-01', end='2025-07-23', interval='1d')
df.index.name = 'timestamp'

# Step 3: Normalize column names (handle MultiIndex)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = ['_'.join(col).strip().lower() for col in df.columns.values]
else:
    df.columns = df.columns.str.lower()

# Step 4: Save raw CSV for reference
df.to_csv('data/spy_historical.csv')

# Step 5: Add technical analysis features
# The KeyError: 'high' occurred because the add_all_ta_features function expects columns named 'open', 'high', 'low', 'close', and 'volume' by default.
# However, your dataframe columns are named 'open_spy', 'high_spy', 'low_spy', 'close_spy', and 'volume_spy'.
# To fix this, I explicitly mapped the function arguments to the correct column names in your dataframe.

df = add_all_ta_features(
    df,
    open='open_spy',
    high='high_spy',
    low='low_spy',
    close='close_spy',
    volume='volume_spy'
)

# Step 6: Add custom rolling features
for window in [5, 10, 20, 50]:
    df[f'rolling_mean_{window}'] = df['close_spy'].rolling(window).mean()
    df[f'rolling_std_{window}'] = df['close_spy'].rolling(window).std()
    df[f'momentum_{window}'] = df['close_spy'] - df['close_spy'].shift(window)
    df[f'rate_of_change_{window}'] = df['close_spy'].pct_change(window)
    df[f'volatility_ratio_{window}'] = df[f'rolling_std_{window}'] / df['close_spy']

# Step 7: Lag features
for lag in [1, 2, 3, 5]:
    df[f'close_lag_{lag}'] = df['close_spy'].shift(lag)
    df[f'close_diff_{lag}'] = df['close_spy'].diff(lag)

# Step 8: Binary classification target
df['target'] = np.where(df['close_spy'].shift(-1) > df['close_spy'], 1, 0)

# Step 9: Drop NA rows created by rolling/lags
df.dropna(inplace=True)

# Step 10: Save processed features
df.to_csv('data/features.csv')

# Optional: Preview output
df.head()


/var/folders/8x/8m0xbdn551jfxk_2552swyj00000gn/T/ipykernel_5677/1270186570.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start='2020-01-01', end='2025-07-23', interval='1d')
[*********************100%***********************]  1 of 1 completed

Note: you may need to restart the kernel to use updated packages.



/Users/aj/futures-bot/venv/lib/python3.11/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/var/folders/8x/8m0xbdn551jfxk_2552swyj00000gn/T/ipykernel_5677/1270186570.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'momentum_{window}'] = df['close_spy'] - df['close_spy'].shift(window)
/var/folders/8x/8m0xbdn551jfxk_2552swyj00000gn/T/ipykernel_5677/1270186570.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

,close_spy,high_spy,low_spy,open_spy,volume_spy,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,volatility_ratio_50,close_lag_1,close_diff_1,close_lag_2,close_diff_2,close_lag_3,close_diff_3,close_lag_5,close_diff_5,target
timestamp,,,,,,,,,,,,,,,,,,,,,
